# Compression ratio of CNN
This notebook is used for calculating the compression ratio of the models

In [1]:
import tensorflow as tf
import tensorflow.keras as keras

In [2]:
model = tf.keras.applications.MobileNetV2(
)

14536120/14536120 [==============================] - 0s 0us/step


Taking a look at the different types of layers

In [20]:
for layer in model.layers:
  print(f"Name {layer.name} \n Type of layer", layer)
  print(f"\ninp:{layer.input_shape}, out:{layer.output_shape}\n\n")

'BatchNormalization'

This looks somewhat as expected

In [44]:
from math import prod
import numpy as np
import plotly.express as px
import pandas as pd

data=[]
for idx, l in enumerate(model.layers[1:-1]):
  data.append((prod(l.output_shape[1:]),type(l).__name__, l.name, idx))
  data.append((prod(l.output_shape[1:]),"all", l.name, idx))

df=pd.DataFrame(data, columns=["features","layer type", "layer name", "idx"])
np.unique([prod(x.output_shape[1:]) for x in model.layers[1:-1]])
fig = px.line(df, x="idx", y="features", color="layer type", custom_data=['layer name'])
fig.update_traces(hovertemplate = "Features:%{y} <br>Name of layer: %{customdata} </br>")
fig.show()


It seems weird that we don't have a linear decline in the features as these spikes. The reasoning could probably be in the paper. Wouldn't it make more sense to have a linear decline.

In [ ]:
import numpy as np
np.unique([str(type(x)) for x in model.layers])

array(["<class 'keras.engine.input_layer.InputLayer'>",
       "<class 'keras.layers.activation.relu.ReLU'>",
       "<class 'keras.layers.convolutional.conv2d.Conv2D'>",
       "<class 'keras.layers.convolutional.depthwise_conv2d.DepthwiseConv2D'>",
       "<class 'keras.layers.core.dense.Dense'>",
       "<class 'keras.layers.merging.add.Add'>",
       "<class 'keras.layers.normalization.batch_normalization.BatchNormalization'>",
       "<class 'keras.layers.pooling.global_average_pooling2d.GlobalAveragePooling2D'>",
       "<class 'keras.layers.reshaping.zero_padding2d.ZeroPadding2D'>"],
      dtype='<U78')

m

Alpha effect on compression

In [47]:
from math import prod
import numpy as np
import plotly.express as px
import pandas as pd

data=[]
for alpha in [0.35,0.5,0.75,1.0,1.3,1.4]:
  model = tf.keras.applications.MobileNetV2(alpha=alpha)
  for idx, l in enumerate(model.layers[1:-1]):
    data.append((prod(l.output_shape[1:]), alpha, idx))

df=pd.DataFrame(data, columns=["features", "alpha", "idx"])
np.unique([prod(x.output_shape[1:]) for x in model.layers[1:-1]])
fig = px.line(df, x="idx", y="features", color="alpha")
fig.show()


25009072/25009072 [==============================] - 0s 0us/step


### Conclusion on compression

We only have to look at the smallet layer which is the output layer

In [50]:
model.layers[0].input_shape

[(None, 224, 224, 3)]

In [52]:
compression = 224*224*3/1280
print(f"Compression ratio is {compression}")

Compression ratio is 117.6
